In [610]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [611]:
res = pd.read_csv('./last/prediction_all.csv')

In [612]:
res.head()

,filename,predict,possibility
0,./origin/all\ATT1000_A5A50BAB-A6EF-4576-A1F8-A...,0,0.599406
1,./origin/all\ATT1002_7DFE65D0-B77A-4C7F-8849-6...,0,0.572508
2,./origin/all\ATT1003_20200718_151437.jpg,0,0.568513
3,./origin/all\ATT1004_Killer Hornet 2020-07-18 ...,0,0.386303
4,./origin/all\ATT1005_Killer Hornet 2020-07-18 ...,0,0.510838


In [613]:
res['filename']=res['filename'].apply(lambda x : x.split('\\')[-1])

In [614]:
res.head()

,filename,predict,possibility
0,ATT1000_A5A50BAB-A6EF-4576-A1F8-A07862AADE3A.jpg,0,0.599406
1,ATT1002_7DFE65D0-B77A-4C7F-8849-685CEA02E862.jpg,0,0.572508
2,ATT1003_20200718_151437.jpg,0,0.568513
3,ATT1004_Killer Hornet 2020-07-18 1.jpg,0,0.386303
4,ATT1005_Killer Hornet 2020-07-18 2.jpg,0,0.510838


In [615]:
res = res.set_index('filename')

In [616]:
res.shape

(3111, 2)

In [617]:
def get_hasimg_csv():
    try:
        hasimg_csv = pd.read_csv('./hasimg.csv')
        hasimg_csv['Detection Date'] = pd.to_datetime(hasimg_csv['Detection Date'])
#         hasimg_csv = hasimg_csv.drop_duplicates('GlobalID')#去除重复
        cls_mapping = {
            'Negative ID': 0,
            'Positive ID': 1,
            'Unprocessed': 2,
            'Unverified': 2
        }

        hasimg_csv['Lab Status']=hasimg_csv['Lab Status'].map(cls_mapping)
        hasimg_csv = hasimg_csv.drop_duplicates('GlobalID')#去除重复
        hasimg_csv.to_csv('./last/hasimg.csv',index=False)
        return hasimg_csv
    except FileNotFoundError:
        img_id_csv = pd.read_excel(img_id_path)
        img_id_csv = img_id_csv[['FileName', 'GlobalID']]
        img_dataset = pd.read_excel(img_dataset_path)
        img_dataset.head()
        pd.Timestamp.min.ceil('D')
        img_dataset = img_dataset[img_dataset['Detection Date'] != '<Null>']
        img_dataset['Detection Date'] = pd.to_datetime(img_dataset['Detection Date'], errors="coerce")
        img_dataset = img_dataset.dropna(subset=['Detection Date'])
        img_dataset['Detection Date']
        img_ds_idindex = img_dataset.set_index('GlobalID')
        img_id_idindex = img_id_csv.set_index('GlobalID')
        hasimg_csv = img_ds_idindex.join(img_id_idindex, how='left')
        hasimg_csv = hasimg_csv.reset_index()
        
        cls_mapping = {
            'Negative ID': 0,
            'Positive ID': 1,
            'Unprocessed': 2,
            'Unverified': 2
        }

        hasimg_csv['Lab Status']=hasimg_csv['Lab Status'].map(cls_mapping)
        hasimg_csv = hasimg_csv.sort_values(by='Detection Date')
        hasimg_csv.to_csv('./data/hasimg.csv')
        return hasimg_csv


In [618]:
hasimg =  get_hasimg_csv()

In [619]:
hasimg.shape

(2051, 11)

In [620]:
hasimg = pd.read_csv('./last/hasimg.csv')

In [621]:
hasimg = hasimg.set_index('FileName')

In [622]:
hasimg=hasimg[['GlobalID','Lab Status']]

In [632]:
hasimg.join(res,how='right').to_csv('./res.csv')

In [629]:
hasimg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2051 entries, ATT747_20200706_105832.jpg to ATT462_49F3CE7F-19DC-4D81-ACBE-ED0916270AB7.jpg
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   GlobalID    2051 non-null   object
 1   Lab Status  2051 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 112.6+ KB


In [630]:
hasimg.columns

Index(['GlobalID', 'Lab Status'], dtype='object')

In [626]:
Unvirefiled = hasimg[hasimg['Lab Status']=='Unverified']

In [627]:
Unvirefiled = Unvirefiled.set_index('FileName')

KeyError: "None of ['FileName'] are in the columns"

In [628]:
Unvirefiled

,GlobalID,Lab Status
FileName,,


In [55]:
Unvirefiled=Unvirefiled[['GlobalID','Lab Status','Latitude','Longitude']]

In [60]:
Unvirefiled.join(res,how='right').to_csv('res.csv')

## Composed

In [633]:
res = pd.read_csv('./res.csv')

In [634]:
res.columns

Index(['filename', 'GlobalID', 'Lab Status', 'predict', 'possibility'], dtype='object')

In [635]:
text = pd.read_excel('./last/text.xlsx')

In [636]:
text.head()

,GlobalID,Detection Date,Notes,Lab Status,Lab Comments,Submission Date,Latitude,Longitude,lenth,note emotion,lab emotion,color,bee lenth
0,{5AC8034E-5B46-4294-85F0-5B13117EBEFE},2019-12-08 00:00:00,"One dead wasp seen in Blaine, and suspect flyi...",Positive ID,,2020-01-15,48.980994,-122.688503,0.213439,0.155823,0.000000,0,0
1,{5EAD3364-2CA7-4A39-9A53-7F9DCF5D2041},2019-10-30 00:00:00,Hornet specimen sent to WSU,Positive ID,,2020-01-15,48.971949,-122.700941,0.102767,0.016162,0.000000,0,0
2,{13B67BCB-AFCE-4100-AD2B-76EF178BA228},2020-01-15 00:00:00,"Massive loss of bees, decapitated. No hornet s...",Unverified,,2020-01-15,48.939200,-122.661300,0.264822,0.098034,0.000000,0,0
3,{124B9BFA-7F7B-4B8E-8A56-42E067F0F72E},2019-09-19 00:00:00,This was the colony that was found and destroy...,Positive ID,"Thanks for adding this, and the great pictures!",2020-02-04,49.149394,-123.943134,0.545455,0.120816,0.062577,0,0
4,{BBBA5BA0-CAFB-43D3-8F1D-FB2D9CF777E0},2019-08-31 00:00:00,I was cleaning my gutters when I heard a snapp...,Unverified,Thanks for this report. I can't verify it from...,2020-02-14,48.723779,-122.354431,0.996047,0.832786,0.421214,0,0


In [637]:
text.columns

Index(['GlobalID', 'Detection Date', 'Notes', 'Lab Status', 'Lab Comments',
       'Submission Date', 'Latitude', 'Longitude', 'lenth', 'note emotion',
       'lab emotion', 'color', 'bee lenth'],
      dtype='object')

In [638]:
text.shape

(4440, 13)

In [639]:
text_index=text.set_index('GlobalID')

In [640]:
res_index = res.set_index('GlobalID')

In [641]:
res_index=res_index[['predict', 'possibility']]

In [642]:
res_index.shape

(3111, 2)

In [643]:
text_index.join(res_index,how='left').columns

Index(['Detection Date', 'Notes', 'Lab Status', 'Lab Comments',
       'Submission Date', 'Latitude', 'Longitude', 'lenth', 'note emotion',
       'lab emotion', 'color', 'bee lenth', 'predict', 'possibility'],
      dtype='object')

In [644]:
total_index = text_index.join(res_index,how='left')

In [645]:
total_file = total_index.reset_index()

In [646]:
what = total_file.groupby('GlobalID').agg('count')['Detection Date']>1
what[what==True]
#无重复

Series([], Name: Detection Date, dtype: bool)

In [647]:
total_file.to_csv('./last/total_report_data.csv')

In [648]:
total_file.rename(columns={'possibility':'CNNpossibility'}, inplace = True) 

In [649]:
total_file.to_csv('./last/total_report_data.csv')

In [650]:
total_file.columns

Index(['GlobalID', 'Detection Date', 'Notes', 'Lab Status', 'Lab Comments',
       'Submission Date', 'Latitude', 'Longitude', 'lenth', 'note emotion',
       'lab emotion', 'color', 'bee lenth', 'predict', 'CNNpossibility'],
      dtype='object')

In [651]:
total_file = total_file[['GlobalID', 'Detection Date', 'Notes', 'Lab Status', 'Lab Comments',
       'Submission Date', 'Latitude', 'Longitude', 'lenth', 'note emotion',
       'lab emotion', 'color', 'bee lenth', 'CNNpossibility']]

In [652]:
total_file.to_csv('total_report_data.csv')

In [653]:
total_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4440 entries, 0 to 4439
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   GlobalID         4440 non-null   object        
 1   Detection Date   4440 non-null   object        
 2   Notes            4440 non-null   object        
 3   Lab Status       4440 non-null   object        
 4   Lab Comments     4438 non-null   object        
 5   Submission Date  4440 non-null   datetime64[ns]
 6   Latitude         4440 non-null   float64       
 7   Longitude        4440 non-null   float64       
 8   lenth            4440 non-null   float64       
 9   note emotion     4440 non-null   float64       
 10  lab emotion      4440 non-null   float64       
 11  color            4440 non-null   int64         
 12  bee lenth        4440 non-null   int64         
 13  CNNpossibility   1992 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(

In [654]:
CNN_mean = total_file['CNNpossibility'].mean()

In [655]:
CNN_mean

0.575819705040699

In [656]:
total_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4440 entries, 0 to 4439
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   GlobalID         4440 non-null   object        
 1   Detection Date   4440 non-null   object        
 2   Notes            4440 non-null   object        
 3   Lab Status       4440 non-null   object        
 4   Lab Comments     4438 non-null   object        
 5   Submission Date  4440 non-null   datetime64[ns]
 6   Latitude         4440 non-null   float64       
 7   Longitude        4440 non-null   float64       
 8   lenth            4440 non-null   float64       
 9   note emotion     4440 non-null   float64       
 10  lab emotion      4440 non-null   float64       
 11  color            4440 non-null   int64         
 12  bee lenth        4440 non-null   int64         
 13  CNNpossibility   1992 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(

In [153]:
del total_file['Season']

In [657]:
new_file = total_file.copy()

In [658]:
new_file['CNNpossibility'].fillna(CNN_mean,inplace=True)

In [659]:
new_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4440 entries, 0 to 4439
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   GlobalID         4440 non-null   object        
 1   Detection Date   4440 non-null   object        
 2   Notes            4440 non-null   object        
 3   Lab Status       4440 non-null   object        
 4   Lab Comments     4438 non-null   object        
 5   Submission Date  4440 non-null   datetime64[ns]
 6   Latitude         4440 non-null   float64       
 7   Longitude        4440 non-null   float64       
 8   lenth            4440 non-null   float64       
 9   note emotion     4440 non-null   float64       
 10  lab emotion      4440 non-null   float64       
 11  color            4440 non-null   int64         
 12  bee lenth        4440 non-null   int64         
 13  CNNpossibility   4440 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(

In [704]:
new_file.to_csv('./last/new.csv')

In [728]:
new_file['Season'] = new_file['Detection Date']
new_file.to_csv('./last/new3.csv')

In [730]:
cls_mapping = {
    'Negative ID': 0,
    'Positive ID': 1,
    'Unprocessed': 2,
    'Unverified': 2
}

new_file['Lab Status']=new_file['Lab Status'].map(cls_mapping)

In [735]:
new_file['Season'] = new_file['Submission Date'].dt.month

In [736]:
new_file.head()

,GlobalID,Detection Date,Notes,Lab Status,Lab Comments,Submission Date,Latitude,Longitude,lenth,note emotion,lab emotion,color,bee lenth,CNNpossibility,Season
0,{0009942D-2F3E-4A2B-9D93-62CD8BA69B8E},2020-08-18 00:00:00,"Saw largest hornet I've ever seen, orange and ...",2,,2020-08-19,47.527105,-122.264096,0.430830,0.273981,0.000000,1,0,0.575820,8
1,{000F804E-2BA9-4838-8E5C-398EE12275E1},2020-04-23 00:00:00,I though it was a queen wasp and looked for a ...,2,,2020-06-12,47.309925,-122.305768,0.288538,0.053153,0.000000,0,0,0.575820,6
2,{003166CE-682C-40AD-9850-8F9CB145B79D},2020-07-07 00:00:00,"Saw this on the side of the road, the entire t...",0,That's a female sawfly. That is her ovipositor...,2020-07-07,48.935934,-122.736701,0.699605,0.060766,0.067496,1,1,0.544641,7
3,{003B9570-B649-4452-93B4-B83025F12014},2020-08-16 00:00:00,"This is a 1.75"" long orange and black wasp and...",2,,2020-08-16,47.602559,-117.453877,0.996047,0.898562,0.000000,1,1,0.575820,8
4,{003BC99B-BC86-4D6A-BD68-9DE4C0271103},2020-08-16 00:00:00,Noticed on my French Drain during watering,2,,2020-08-17,47.323825,-122.372725,0.162055,0.033232,0.000000,0,0,0.575820,8


In [737]:
new_file.to_csv('./last/best.csv')

In [710]:
a =pd.read_csv('./last/new3.csv')

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 20270-20271: invalid continuation byte

春天345- 0  
夏天678- 1  
秋天9 10 11- 2  
冬天 12 1 2 -3  

In [684]:
new_file.columns

Index(['GlobalID', 'Detection Date', 'Notes', 'Lab Status', 'Lab Comments',
       'Submission Date', 'Latitude', 'Longitude', 'lenth', 'note emotion',
       'lab emotion', 'color', 'bee lenth', 'CNNpossibility'],
      dtype='object')

In [685]:
my_data=new_file[['GlobalID','Detection Date','Latitude','Longitude','lenth','note emotion','lab emotion','color', 'bee lenth', 'CNNpossibility','Lab Status']]

In [686]:
my_data.columns

Index(['GlobalID', 'Detection Date', 'Latitude', 'Longitude', 'lenth',
       'note emotion', 'lab emotion', 'color', 'bee lenth', 'CNNpossibility',
       'Lab Status'],
      dtype='object')

In [687]:
data = my_data[['GlobalID','Detection Date','Latitude','Longitude','lenth','note emotion','lab emotion','color', 'bee lenth', 'CNNpossibility']]

In [688]:
data  = data.set_index('GlobalID')

In [689]:
data=data.reset_index()

In [695]:
data.to_csv('./last/new2.csv')

In [696]:
pd.read_csv('./last/new2.csv')

,Unnamed: 0,GlobalID,Detection Date,Latitude,Longitude,lenth,note emotion,lab emotion,color,bee lenth,CNNpossibility,Season
0,0,{0009942D-2F3E-4A2B-9D93-62CD8BA69B8E},2020/8/18 0:00,47.527105,-122.264096,0.430830,0.273981,0.000000,1,0,0.575820,08
1,1,{000F804E-2BA9-4838-8E5C-398EE12275E1},2020/4/23 0:00,47.309925,-122.305768,0.288538,0.053153,0.000000,0,0,0.575820,04
2,2,{003166CE-682C-40AD-9850-8F9CB145B79D},2020/7/7 0:00,48.935934,-122.736701,0.699605,0.060766,0.067496,1,1,0.544641,07
3,3,{003B9570-B649-4452-93B4-B83025F12014},2020/8/16 0:00,47.602559,-117.453877,0.996047,0.898562,0.000000,1,1,0.575820,08
4,4,{003BC99B-BC86-4D6A-BD68-9DE4C0271103},2020/8/16 0:00,47.323825,-122.372725,0.162055,0.033232,0.000000,0,0,0.575820,08
...,...,...,...,...,...,...,...,...,...,...,...,...
4435,4435,{FFA96EE6-153C-4E14-B314-93DA2A4FAD55},2020/8/1 0:00,48.035730,-122.773200,1.000000,0.007733,0.000000,1,1,0.575820,08
4436,4436,{FFAB9655-A203-47D6-BAFB-BFB0818E7CDD},2020/6/14 0:00,47.287735,-116.936844,0.446640,0.348076,0.353948,0,1,0.714613,06
4437,4437,{FFCF73D5-EC41-4EE3-A3A6-2DCD831E4D92},2020/5/17 0:00,47.436259,-122.038348,0.992095,0.092946,0.000000,1,1,0.575820,05
4438,4438,{FFD28418-72D8-4F9F-8A52-86A26874D4F2},2020/9/11 0:00,47.738572,-122.339470,0.193676,0.175882,0.000000,0,0,0.575820,09


In [691]:
SeasonDict=[0,4,4,1,1,1,2,2,2,3,3,3,4]

In [693]:
data['Season']= data['Detection Date']

In [327]:
a=lis.apply(lambda x:x)

In [329]:
import datetime i

In [334]:
[ int(str(i.strftime("%Y-%m-%d %H:%M:%S").split('-')[1])) for i in lis]

AttributeError: 'str' object has no attribute 'strftime'

In [288]:
np.datetime64(lis)

ValueError: Could not convert object to NumPy datetime

In [259]:
data['Detection Date'] = np.datetime64(data['Detection Date'])

ValueError: Could not convert object to NumPy datetime

In [231]:
label = my_data[['GlobalID','Lab Status']]

In [170]:
label.set_index('GlobalID')

,Lab Status
GlobalID,
{0009942D-2F3E-4A2B-9D93-62CD8BA69B8E},Unverified
{000F804E-2BA9-4838-8E5C-398EE12275E1},Unverified
{003166CE-682C-40AD-9850-8F9CB145B79D},Negative ID
{003B9570-B649-4452-93B4-B83025F12014},Unverified
{003BC99B-BC86-4D6A-BD68-9DE4C0271103},Unverified
...,...
{FFA96EE6-153C-4E14-B314-93DA2A4FAD55},Unverified
{FFAB9655-A203-47D6-BAFB-BFB0818E7CDD},Negative ID
{FFCF73D5-EC41-4EE3-A3A6-2DCD831E4D92},Unverified


In [739]:
import pandas as pd
my_data2 = pd.read_csv('./last/best.csv')

In [741]:
my_data2.shape

(4440, 16)

In [742]:
my_data2.head()

,Unnamed: 0,GlobalID,Detection Date,Notes,Lab Status,Lab Comments,Submission Date,Latitude,Longitude,lenth,note emotion,lab emotion,color,bee lenth,CNNpossibility,Season
0,0,{0009942D-2F3E-4A2B-9D93-62CD8BA69B8E},2020-08-18 00:00:00,"Saw largest hornet I've ever seen, orange and ...",2,,2020-08-19,47.527105,-122.264096,0.430830,0.273981,0.000000,1,0,0.575820,8
1,1,{000F804E-2BA9-4838-8E5C-398EE12275E1},2020-04-23 00:00:00,I though it was a queen wasp and looked for a ...,2,,2020-06-12,47.309925,-122.305768,0.288538,0.053153,0.000000,0,0,0.575820,6
2,2,{003166CE-682C-40AD-9850-8F9CB145B79D},2020-07-07 00:00:00,"Saw this on the side of the road, the entire t...",0,That's a female sawfly. That is her ovipositor...,2020-07-07,48.935934,-122.736701,0.699605,0.060766,0.067496,1,1,0.544641,7
3,3,{003B9570-B649-4452-93B4-B83025F12014},2020-08-16 00:00:00,"This is a 1.75"" long orange and black wasp and...",2,,2020-08-16,47.602559,-117.453877,0.996047,0.898562,0.000000,1,1,0.575820,8
4,4,{003BC99B-BC86-4D6A-BD68-9DE4C0271103},2020-08-16 00:00:00,Noticed on my French Drain during watering,2,,2020-08-17,47.323825,-122.372725,0.162055,0.033232,0.000000,0,0,0.575820,8


In [474]:
ds = my_data2.copy()

In [475]:
ds.rename(columns={'Month':'Season'},inplace=True) 

In [476]:
ds.head(2)

,Unnamed: 0,GlobalID,Detection Date,Season,Lab Status,Lab Comments,Submission Date,Latitude,Longitude,lenth,note emotion,lab emotion,color,bee lenth,predict,CNNpossibility,Unnamed: 16
0,0,{0009942D-2F3E-4A2B-9D93-62CD8BA69B8E},2020/8/18 0:00,1.0,Unverified,,2020/8/19,47.527105,-122.264096,0.43083004,0.273981,0.0,1.0,0.0,NaN,NaN,NaN
1,1,{000F804E-2BA9-4838-8E5C-398EE12275E1},2020/4/23 0:00,0.0,Unverified,,2020/6/12,47.309925,-122.305768,0.288537549,0.053153,0.0,0.0,0.0,NaN,NaN,NaN


In [480]:
ds.columns

Index(['Unnamed: 0', 'GlobalID', 'Detection Date', 'Season', 'Lab Status',
       'Lab Comments', 'Submission Date', 'Latitude', 'Longitude', 'lenth',
       'note emotion', 'lab emotion', 'color', 'bee lenth', 'predict',
       'CNNpossibility', 'Unnamed: 16'],
      dtype='object')

In [481]:
ds = ds[['GlobalID', 'Detection Date', 'Season', 'Lab Status',
       'Lab Comments', 'Submission Date', 'Latitude', 'Longitude', 'lenth',
       'note emotion', 'lab emotion', 'color', 'bee lenth', 'predict',
       'CNNpossibility']]

In [486]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4437 entries, 0 to 4436
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   GlobalID         4437 non-null   object 
 1   Detection Date   4437 non-null   object 
 2   Season           4423 non-null   float64
 3   Lab Status       4437 non-null   object 
 4   Lab Comments     4435 non-null   object 
 5   Submission Date  4437 non-null   object 
 6   Latitude         4437 non-null   object 
 7   Longitude        4437 non-null   object 
 8   lenth            4437 non-null   object 
 9   note emotion     4437 non-null   float64
 10  lab emotion      4437 non-null   float64
 11  color            4437 non-null   float64
 12  bee lenth        4434 non-null   float64
 13  predict          55 non-null     float64
 14  CNNpossibility   54 non-null     float64
dtypes: float64(7), object(8)
memory usage: 554.6+ KB


In [488]:
def changelabel(x):
    if x=='Positive ID':
        return 1
    elif x=='Negative ID':
        return 0
    else:
        return 2

In [489]:
ds['Lab Status'] = ds['Lab Status'].apply(lambda x:changelabel(x))

In [492]:
del ds['predict']

In [493]:
CNN_mean

0.3755471482263779

In [ ]:
ds['CNNpossibility']

## start

In [755]:
final = pd.read_csv('./last/best.csv')

In [876]:
final = pd.read_csv('./last/best.csv')

In [885]:
best = final[(final['Lab Status']==1) | (final['Lab Status']==0)]

In [889]:
best.to_csv('./last/train.csv')

In [890]:
test = final[(final['Lab Status']==2) ]

In [891]:
test.to_csv('./last/test.csv')

In [756]:
train=final[(final['Lab Status']==0) | (final['Lab Status']==1) ]

In [757]:
train.columns

Index(['Unnamed: 0', 'GlobalID', 'Detection Date', 'Notes', 'Lab Status',
       'Lab Comments', 'Submission Date', 'Latitude', 'Longitude', 'lenth',
       'note emotion', 'lab emotion', 'color', 'bee lenth', 'CNNpossibility',
       'Season'],
      dtype='object')

In [771]:
final['Season'].max()

10

In [773]:
data=train[['Latitude', 'Longitude', 'lenth','note emotion', 'lab emotion', 'color', 'bee lenth', 'CNNpossibility',
       'Season']]

In [774]:
label = train[['Lab Status']]

In [844]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.4, random_state=0)

In [845]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [857]:
from imblearn.over_sampling import SMOTE, ADASYN

X_resampled_smote, y_resampled_smote = SMOTE(k_neighbors=5).fit_sample(X_train,y_train)

## origin

In [859]:
model = RandomForestClassifier()
model.fit(X_resampled_smote, y_resampled_smote)
result = model.score(X_test,y_test)
result

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.9892086330935251

In [860]:
model.predict_proba(X_test)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [861]:
import numpy as np
from sklearn import svm
 

clt = svm.SVC(probability = True)
clt.fit(X_resampled, y_resampled)
clt.score(X_test,y_test)
# clt.predict(X_test)

E:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.9280575539568345

In [862]:
clt.predict_proba(X_test)

array([[0.88467794, 0.11532206],
       [0.17455427, 0.82544573],
       [0.821588  , 0.178412  ],
       ...,
       [0.95146668, 0.04853332],
       [0.57900387, 0.42099613],
       [0.32471843, 0.67528157]])

In [863]:
from sklearn import linear_model

In [864]:
lr=linear_model.LogisticRegression(solver='liblinear')#在新版的sklearn当中只需要指定后面的参数值就不会进行报错啦！
lr.fit(X_resampled,y_resampled)

E:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression(solver='liblinear')

In [865]:
lr.score(X_test,y_test)

0.9028776978417267

In [866]:
lr.predict_proba(X_test)

array([[4.86938248e-01, 5.13061752e-01],
       [9.99716002e-01, 2.83997727e-04],
       [8.41879342e-01, 1.58120658e-01],
       ...,
       [9.67205815e-01, 3.27941850e-02],
       [1.41708833e-01, 8.58291167e-01],
       [9.63978949e-01, 3.60210509e-02]])

In [868]:
from sklearn import tree

In [ ]:
tree_clf = tree.DecisionTreeClassifier()
tree_clf = tree_clf.fit(X_resampled,y_resampled_smote)
tree_clf.predict(X_test)

In [871]:
tree_clf

In [892]:
import matplotlib.pyplot as plt